### Intro to PyTorch - Udacity

1. `Reshaping Tensors`
2. `Numpy to Torch and back`
3. `Getting Datasets`
4. `2-Layer NN from SCRATCH`
5. `Softmax from SCRATCH`
    * Broadcasting in Numpy INTUITION
    
    
6. `Creating ready-made NN( inheriting torch.nn.Module class )`
7. `Creating ready-made NN( using nn.Sequential )`

### 1. Reshaping Tensors

Let **weights** be a tensor

* `weights.reshape(a, b)` will return a new tensor with the same data(points to the same memory location) as `weights` with size `(a, b)` sometimes, and sometimes a clone, as in it copies the data to another part of memory.
* `weights.resize_(a, b)` returns the same tensor with a different shape. However, if the new shape results in fewer elements than the original tensor, some elements will be removed from the tensor (but not from memory). If the new shape results in more elements than the original tensor, new elements will be uninitialized in memory. Here I should note that the underscore at the end of the method denotes that this method is performed **in-place**. Here is a great forum thread to [read more about in-place operations](https://discuss.pytorch.org/t/what-is-in-place-operation/16244) in PyTorch.
* `weights.view(a, b)` will return a new tensor with the same data as `weights` with size `(a, b)`.


In [8]:
import torch

x = torch.randn(1)
y = torch.randn(1)

print(x, y)
print(f'id(x): {id(x)}, id(y): {id(y)}')

tensor([0.8947]) tensor([-0.3227])
id(x): 2204467581432, id(y): 2204497406712


In [9]:
x = x + y # Normal operation 
print(f'x = {x} id(x) = {id(x)}') # New location for x

x = tensor([0.5720]) id(x) = 2204497407864


In [10]:
x += y # inplace operation
print(f'x = {x} id(x) = {id(x)}') # existing location used(in-place)

x = tensor([0.2493]) id(x) = 2204497407864


In [11]:
x.add_(y) # inplace operation
print(f'x = {x} id(x) = {id(x)}') # existing location used(in-place)

x = tensor([-0.0734]) id(x) = 2204497407864


**Inplace operations in pytorch are always postfixed with a _ , like .add_() or .scatter_(). Python operations like += or *= are also inplace operations.**
<hr>

### 2. Numpy to Torch and back

Special bonus section! PyTorch has a great feature for converting between Numpy arrays and Torch tensors. To create a tensor from a Numpy array, use `torch.from_numpy()`. To convert a tensor to a Numpy array, use the `.numpy()` method.

In [12]:
import numpy as np
a = np.random.rand(4,3)
a

array([[0.62610611, 0.73824629, 0.68162487],
       [0.45799142, 0.81512616, 0.9046208 ],
       [0.13016165, 0.132936  , 0.03448641],
       [0.46551398, 0.73371913, 0.05358193]])

In [13]:
b = torch.from_numpy(a)
b

tensor([[0.6261, 0.7382, 0.6816],
        [0.4580, 0.8151, 0.9046],
        [0.1302, 0.1329, 0.0345],
        [0.4655, 0.7337, 0.0536]], dtype=torch.float64)

In [14]:
b.numpy()

array([[0.62610611, 0.73824629, 0.68162487],
       [0.45799142, 0.81512616, 0.9046208 ],
       [0.13016165, 0.132936  , 0.03448641],
       [0.46551398, 0.73371913, 0.05358193]])

The memory is shared between the Numpy array and Torch tensor, so if you change the values in-place of one object, the other will change as well.

In [15]:
# Multiply PyTorch Tensor by 2, in place
b.mul_(2)

tensor([[1.2522, 1.4765, 1.3632],
        [0.9160, 1.6303, 1.8092],
        [0.2603, 0.2659, 0.0690],
        [0.9310, 1.4674, 0.1072]], dtype=torch.float64)

In [16]:
# Numpy array matches new values from Tensor
a

array([[1.25221221, 1.47649258, 1.36324974],
       [0.91598285, 1.63025232, 1.8092416 ],
       [0.26032331, 0.26587199, 0.06897282],
       [0.93102797, 1.46743827, 0.10716386]])

<hr>

### 3. Getting datasets

In [17]:
from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training data
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

We have the training data loaded into `trainloader` and we make that an iterator with `iter(trainloader)`. Later, we'll use this to loop through the dataset for training, like

```python
for image, label in trainloader:
    ## do things with images and labels
```

You'll notice I created the `trainloader` with a batch size of 64, and `shuffle=True`. The batch size is the number of images we get in one iteration from the data loader and pass through our network, often called a *batch*. And `shuffle=True` tells it to shuffle the dataset every time we start going through the data loader again. But here I'm just grabbing the first batch so we can check out the data. We can see below that `images` is just a tensor with size `(64, 1, 28, 28)`. So, 64 images per batch, 1 color channel, and 28x28 images.

In [18]:
dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)

<class 'torch.Tensor'>
torch.Size([64, 1, 28, 28])
torch.Size([64])


In [19]:
inputs = images.view(images.shape[0], -1)
inputs.shape

torch.Size([64, 784])

### 4. 2-Layer NN from SCRATCH

Build a multi-layer network with 784 input units, 256 hidden units, and 10 output units using random tensors for the weights and biases. For now, use a sigmoid activation for the hidden layer and leave the output without one.

In [20]:
## Your solution
np.random.seed(7)

def activation(x):
    return 1 / (1 + torch.exp(-x)) # or np.exp(-x)

# Flatten the input image
inputs = images.view(images.shape[0], -1)# or images.view(64, 784)

# Set network layer sizes
n_input = inputs.shape[1] # 784
n_hidden = 256
n_output = 10

# Create parameters
W1 = torch.randn(n_input, n_hidden) # weights for the hidden layer- 784X256
W2 = torch.randn(n_hidden, n_output) # weights for the output layer- 256X10

B1 = torch.randn(1, n_hidden) # biases for the hidden layer- 1X256
B2 = torch.randn(1, n_output) # biases for the output layer- 1X10

# output of hidden layer- 64X256 ie 1X256 values for 64 images in the batch
# torch.mm(inputs, W1)- 64X256
# even though B1 is of shape 1X256, it broadcasts to 64X256 when made to add with a 64X256 matrix
h = activation(torch.mm(inputs, W1) + B1) 

# torch.mm(h, W2)- 64X10
# B2 broadcasts from 1X10 to 64X10 on addition with a 64X10 matrix
out = torch.mm(h, W2) + B2 # network output- 64X10 ie 10 output values for each of 64 images

print(out.shape)
print(out)

torch.Size([64, 10])
tensor([[ 1.2275e+00,  1.7868e+01, -3.3610e+00,  9.6572e+00, -4.4323e+00,
          2.3538e+00, -1.7093e+01,  1.1102e+01,  7.5253e+00,  6.0052e+00],
        [ 1.9197e+00,  3.9145e+00, -4.1184e+00,  1.3343e+01, -1.7035e+01,
         -7.0955e+00, -2.2273e+01,  6.2673e+00, -9.8147e-01,  1.0414e+01],
        [ 8.1331e+00,  1.6488e+01, -2.4356e-01,  7.5493e+00, -7.8329e+00,
          7.8338e+00, -9.8890e+00, -3.9259e+00,  9.7671e+00, -1.7121e+00],
        [-8.2881e+00,  8.3918e+00,  2.4722e+00,  1.5421e+01, -5.5535e+00,
         -1.6099e+01, -1.6919e+01,  8.2451e+00,  7.3715e+00,  6.4537e+00],
        [-3.0707e+00,  1.1511e+01,  5.7331e+00,  9.1911e+00, -1.1015e+01,
         -1.7204e+01, -1.1893e+01,  1.2052e+01,  5.1869e+00,  5.8618e+00],
        [ 8.3273e+00,  2.2861e+01,  5.9417e+00,  4.5867e+00, -8.6057e-01,
          4.6767e-01, -5.5814e+00,  2.7008e+00,  5.5566e+00, -1.2963e+00],
        [ 6.5105e+00,  1.0396e+01,  1.5333e+00,  9.1974e+00, -1.2533e+01,
         -5

          3.5082e+00, -1.5746e+01, -4.7840e+00,  9.5252e+00, -7.5188e+00]])


### 5. Softmax from SCRATCH

In [21]:
def softmax(x):
    num = torch.exp(x)
    den = torch.sum(torch.exp(x), dim=1).view(-1, 1) # change shape from (64, ) to (64, 1).
    return num/den

# Here, out should be the output of the network in the previous excercise with shape (64,10)
probabilities = softmax(out)

# Does it have the right shape? Should be (64, 10)
print(probabilities.shape)
# Does it sum to 1?
print(probabilities.sum(dim=1))

torch.Size([64, 10])
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])


<img src="../img/softmax-formula.png" width="360px">

The softmax function is calculated by:

1. Find the exponential of the quantity whose softmax score we need to know( `Numerator` )
2. Find the exponential of all the quantities in the group over which we need to find the softmax score( the value of each of the 10 output nodes corresponding to 10 class labels in our case ) and add( $\Sigma_j$ ) it ( `Denominator` )

ie for each value in the group, calculate its exponential and divide it by the sum of exponentials of all values in its group

In our case, we are calculating softmax for the output of all 64 images together. 

Let's start out by viewing how we calculate softmax for a single image.

In [30]:
# from our NN output of 64 images, get output of the first image
# out.shape == [64, 10]
single_image_output = out[0] # or out[0, :] as we require all the 10 column values in the first image

print(single_image_output.shape)
print(single_image_output)

torch.Size([10])
tensor([  1.2275,  17.8675,  -3.3610,   9.6572,  -4.4323,   2.3538, -17.0929,
         11.1017,   7.5253,   6.0052])


In [31]:
# now we can find the softmax scores of the values in the output

# 1. Numerator
# instead of finding the exponential of each of the values in the output, we can find exponential of all values in the
# output using vectorised operations
numerator = torch.exp(single_image_output)

print(numerator.shape)
print(numerator)

torch.Size([10])
tensor([3.4127e+00, 5.7512e+07, 3.4701e-02, 1.5635e+04, 1.1887e-02, 1.0525e+01,
        3.7727e-08, 6.6287e+04, 1.8543e+03, 4.0552e+02])


In [33]:
# 2. Denominator
# we need to find the sum of exponentials of all values in the output
exponentials = torch.exp(single_image_output) # exponential of all values in the output as a tensor
denominator = torch.sum(exponentials) # add all the exponential values to get the sum

print(denominator.shape)
print(denominator)

torch.Size([])
tensor(57596524.)


In [46]:
# Find softmax

softmax = numerator / denominator

# By dividing the numerator tensor of size 10( 1 dimension ) with denominator tensor of size 1 ( 0 dimension or scalar )
# the denominator gets broadcasted to a suitable shape, here to size 10 to make the division happen

# ie by running numerator / denominator actually denominator gets modified to
broadcasted_denominator = torch.Tensor(10).fill_(denominator)

print(f'Broadcasted denominator: {broadcasted_denominator} of shape {broadcasted_denominator.shape}\n')

softmax_broadcasted = numerator / broadcasted_denominator

print(f'Softmax: {softmax}')
print(f'Softmax using denominator of same shape as numerator: {softmax_broadcasted}\n')


equality_check = torch.all(torch.eq(softmax, softmax_broadcasted))
if equality_check == 1:
    print('Both tensors are of same value')
else:
    print('The tensors differ')

Broadcasted denominator: tensor([57596524., 57596524., 57596524., 57596524., 57596524., 57596524.,
        57596524., 57596524., 57596524., 57596524.]) of shape torch.Size([10])

Softmax: tensor([5.9251e-08, 9.9854e-01, 6.0249e-10, 2.7145e-04, 2.0639e-10, 1.8274e-07,
        6.5501e-16, 1.1509e-03, 3.2195e-05, 7.0407e-06])
Softmax using denominator of same shape as numerator: tensor([5.9251e-08, 9.9854e-01, 6.0249e-10, 2.7145e-04, 2.0639e-10, 1.8274e-07,
        6.5501e-16, 1.1509e-03, 3.2195e-05, 7.0407e-06])

Both tensors are of same value


**The takeaway is that given 10 values to softmax, we use all these 10 values to calculate new 10 values**

Now let's get on to calculating softmax over the batch of 64 images
<img src="../img/softmax-formula.png" width="360px">

In [47]:
print(out.shape) # we have 10 class labels each for the 64 images

torch.Size([64, 10])


In [49]:
# 1. Numerator
# Find the exponential of all values we have
numerator = torch.exp(out)
print(num.shape)

torch.Size([64, 10])


In [56]:
# 2. Denominator
# find exponentials
expos = torch.exp(out)
# we need to add the 10 exponentials of each image and 
# get a new tensor with a sum for each image ie a 1D tensor of 64 values
denominator = torch.sum(expos, dim=1) # dim=1 adds up all values in a row, ie for each image
print(denominator.shape)

torch.Size([64])


expos is a `[64, 10]` tensor. `dim=1` in the `torch.sum` function tells it to sum over the values in the second dimension ( first dimenstion is specified by dim=0 ). This results in a `[64]` tensor ie a 1D tensor. 

If we had given `dim=0` as the argument, the returned tensor would had a shape of `[10]`, as the values in the first dimension would be added up.

Incase you are wondering what would happen, if we omit the `dim` argument, it would just sum up all the values in the tensor.

**torch.Size([64])** ~ **numpy (64, )** it is considered to be **(1, 64)** during broadcasting.

So if we divide (64, 10) the numerator with (64, ), python will try to broadcast to make the tensors of same size.
(64, ) to (1, 64) to ?

(64, 10) and (1, 64) can't be broadcasted to equal size and python outputs error.

Now if we reshape (64, ) to (64, 1), the broadcasting goes like this,
(64, 1) to (64, 10)

Now, (64, 10) the numerator and (64, 10) the broadcasted denominator can be divided

<hr>

### 6. Creating ready-made NN( inheriting torch.nn.Module class )

In [23]:
## Your solution here
from torch import nn
import torch.nn.functional as f

# 3 layer NN
class Network3L(nn.Module):
    def __init__(self):
        super().__init__()

        # first fc hidden layer
        self.fc1 = nn.Linear(784, 128)

        # second fc hidden layer
        self.fc2 = nn.Linear(128, 64)

        # fc output layer
        self.fc3 = nn.Linear(64, 10)
    
    def forward(self, x):
        h1 = F.relu(self.fc1(x)) # output of first hidden layer
        h2 = F.relu(self.fc2(h1)) # output of second hidden layer
        out = F.sigmoid(self.fc3(h2)) # neural network output
        
model = Network3L()
model

Network3L(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)

In [24]:
print(model.fc1.weight.shape, model.fc1.bias.shape)

torch.Size([128, 784]) torch.Size([128])


<hr>

### 7. Creating ready-made NN( using ***nn.Sequential*** )

PyTorch provides a convenient way to build networks like this where a tensor is passed sequentially through operations, `nn.Sequential` ([documentation](https://pytorch.org/docs/master/nn.html#torch.nn.Sequential)). Using this to build the equivalent network:

In [27]:
# Hyperparameters for our network
input_size = 784
hidden_sizes = [128, 64]
output_size = 10

# Build a feed-forward network
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size),
                      nn.Softmax(dim=1))
print(model)

# Forward pass through the network and display output
images, labels = next(iter(trainloader)) # get first batch of images and their labels
images.resize_(images.shape[0], 1, 784) # resize images in-place using resize_
ps = model.forward(images[0,:]) # forward pass through our NN, the batch of images

Sequential(
  (0): Linear(in_features=784, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=64, bias=True)
  (3): ReLU()
  (4): Linear(in_features=64, out_features=10, bias=True)
  (5): Softmax()
)


In [26]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(6, 9))

ax[0].imshow(images[0].view(28, 28))
ax[0].axis('off')

ax[1].barh(np.arange(10), ps.data.numpy().squeeze())
ax[1].set_aspect(0.1)
ax[1].set_xlim(0, 1.1)
ax[1].set_yticks(np.arange(10))
ax[1].set_yticklabels(np.arange(10))
ax[1].set_title('Class Probability')

plt.tight_layout()
#helper.view_classify(images[0].view(1, 28, 28), ps)

<hr>